In [22]:
import gcp
import utils

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/joe/Dev/key/dauntless-graph-293915-0a54f42271b4.json"

In [4]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [5]:
implicit()

[]


In [6]:
c = gcp.get_compute_client()

In [7]:
print(c)

In [8]:
region = "us-east1"
zone = gcp.get_zone_for_region(region)

In [9]:
zone

'us-east1-b'

In [10]:
c = gcp.get_compute_client().instances()

In [11]:
PROJECT = "dauntless-graph-293915"

In [16]:
name = "gpu-isness-server-sc-p100-2"	
i = gcp.get_instance(region, name)


In [17]:
i

{'narupaStatus': False, 'status': 'UNKNOWN'}

In [14]:
response = gcp.get_compute_client().instances().get(project=PROJECT, zone=zone, instance=name).execute()

In [15]:
response

{'id': '325030160619962431',
 'creationTimestamp': '2020-11-07T16:41:52.425-08:00',
 'name': 'gpu-isness-server-sc-p100-2',
 'description': '',
 'tags': {'items': ['http-server', 'https-server'],
  'fingerprint': '6smc4R4d39I='},
 'machineType': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/machineTypes/n1-standard-8',
 'status': 'RUNNING',
 'zone': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b',
 'canIpForward': False,
 'networkInterfaces': [{'network': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/global/networks/default',
   'subnetwork': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/regions/us-east1/subnetworks/default',
   'networkIP': '10.142.0.21',
   'name': 'nic0',
   'accessConfigs': [{'type': 'ONE_TO_ONE_NAT',
     'name': 'External NAT',
     'natIP': '35.229.127.251',
     'networkTier': 'PREMIUM',
     'kind': 'compute#accessConfig'}],
   '

In [19]:
ip = response['networkInterfaces'][0]['accessConfigs'][0]['natIP']
response['instanceIp'] = ip
response['narupaStatus'] = gcp.get_narupa_status(ip)

In [20]:
response['narupaStatus']

False

In [21]:
response['instanceIp']

'35.229.127.251'

In [53]:
def create_instance(region, duration, simulation=None, topology=None, trajectory=None):
    zone = gcp.get_zone_for_region(region)

    machine_type = 'n1-standard-1'
    gpu_type = 'nvidia-tesla-p100'
    disk_size_gb = '32'

    name = 'anuma-isness-{}'.format(utils.generate_short_id())
    metadata = [
        { 'key': 'google-logging-enabled', 'value': 'true' },
        { 'key': 'duration', 'value': duration},
        { 'key': 'startup-script', 'value': '#!/bin/bash\n'}
    ]

    config = {
        'name': name,
        'zone': 'projects/{}/zones/{}'.format(PROJECT, zone),
        'machineType': 'projects/{}/zones/{}/machineTypes/{}'.format(PROJECT, zone, machine_type),
        'displayDevice': {
            'enableDisplay': False
        },
        'metadata': {
            'items': metadata
        },
        'tags': {
            'items': [ 'narupa-simulation' ]
        },
        'guestAccelerators': [
            {
            'acceleratorCount': 1,
            'acceleratorType': 'projects/{}/zones/{}/acceleratorTypes/{}'.format(PROJECT, zone, gpu_type)
            }
        ],
        #'sourceMachineImage': 'projects/dauntless-graph-293915/global/machineImages/nvidia-gpu-sc-2',
        'disks': [
            {
            'type': 'PERSISTENT',
            'boot': True,
            'mode': 'READ_WRITE',
            'autoDelete': True,
            'deviceName': name,
            'initializeParams': {
                'sourceImage': 'projects/nvidia-ngc-public/global/images/nvidia-gpu-cloud-image-20200629',
                'diskType': 'projects/{}/zones/{}/diskTypes/pd-standard'.format(PROJECT, zone),
                'diskSizeGb': disk_size_gb
            },
            'diskEncryptionKey': {}
            }
        ],
        'canIpForward': False,
        'networkInterfaces': [
            {
            'subnetwork': 'projects/{}/regions/{}/subnetworks/default'.format(PROJECT, region),
            'accessConfigs': [
                {
                'name': 'External NAT',
                'type': 'ONE_TO_ONE_NAT',
                'networkTier': 'PREMIUM'
                }
            ]
            }
        ],
        'serviceAccounts': [
            {
            'email': '669110238078-compute@developer.gserviceaccount.com',
            'scopes': [
                'https://www.googleapis.com/auth/cloud-platform',
                'https://www.googleapis.com/auth/devstorage.read_only',
                'https://www.googleapis.com/auth/logging.write',
                'https://www.googleapis.com/auth/monitoring.write',
                'https://www.googleapis.com/auth/servicecontrol',
                'https://www.googleapis.com/auth/service.management.readonly',
                'https://www.googleapis.com/auth/trace.append'
            ]
            }
        ],
        'scheduling': {
            'preemptible': False,
            'onHostMaintenance': 'TERMINATE',
            'automaticRestart': False
        },
        'deletionProtection': False,
        'shieldedInstanceConfig': {
            'enableSecureBoot': False,
            'enableVtpm': True,
            'enableIntegrityMonitoring': True
        }
    }

    response = gcp.get_compute_client().instances().insert(project=PROJECT, zone=zone, body=config).execute()
    response['instanceName'] = name
    return response


In [24]:
#dauntless-graph-293915/zones/us-east1-b/instances/

In [25]:
duration = 30

In [54]:
create_instance(region, duration, simulation=None, topology=None, trajectory=None)

HttpError: <HttpError 400 when requesting https://compute.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/instances?alt=json returned "Invalid value for field 'resource.disks': ''. No disks are specified.">

In [35]:
 #669110238078-compute@developer.gserviceaccount.com

In [36]:
response = gcp.get_compute_client().instances().get(project=PROJECT, zone=zone, instance=name).execute()

In [37]:
response 

{'id': '325030160619962431',
 'creationTimestamp': '2020-11-07T16:41:52.425-08:00',
 'name': 'gpu-isness-server-sc-p100-2',
 'description': '',
 'tags': {'items': ['http-server', 'https-server'],
  'fingerprint': '6smc4R4d39I='},
 'machineType': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/machineTypes/n1-standard-8',
 'status': 'RUNNING',
 'zone': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b',
 'canIpForward': False,
 'networkInterfaces': [{'network': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/global/networks/default',
   'subnetwork': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/regions/us-east1/subnetworks/default',
   'networkIP': '10.142.0.21',
   'name': 'nic0',
   'accessConfigs': [{'type': 'ONE_TO_ONE_NAT',
     'name': 'External NAT',
     'natIP': '35.229.127.251',
     'networkTier': 'PREMIUM',
     'kind': 'compute#accessConfig'}],
   '

In [38]:
name

'gpu-isness-server-sc-p100-2'

In [45]:
name = "narupa-simulation-ezqobh"
response = gcp.get_compute_client().instances().get(project=PROJECT, zone=zone, instance=name).execute()

In [46]:
response

{'id': '5526523580877542968',
 'creationTimestamp': '2020-11-10T15:57:11.680-08:00',
 'name': 'narupa-simulation-ezqobh',
 'tags': {'items': ['narupa-simulation'], 'fingerprint': 'XixrPlRL6Kk='},
 'machineType': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/machineTypes/n1-standard-1',
 'status': 'RUNNING',
 'zone': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b',
 'canIpForward': False,
 'networkInterfaces': [{'network': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/global/networks/default',
   'subnetwork': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/regions/us-east1/subnetworks/default',
   'networkIP': '10.142.0.23',
   'name': 'nic0',
   'accessConfigs': [{'type': 'ONE_TO_ONE_NAT',
     'name': 'External NAT',
     'natIP': '104.196.55.110',
     'networkTier': 'PREMIUM',
     'kind': 'compute#accessConfig'}],
   'fingerprint': 'C5o3G1iI0OE=',
   '

In [ ]:
his is the REST request with the parameters you have selected.
POST https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/instances
{
  "kind": "compute#instance",
  "name": "nvidia-gpu-sc-1",
  "zone": "projects/dauntless-graph-293915/zones/us-east1-b",
  "minCpuPlatform": "Automatic",
  "machineType": "projects/dauntless-graph-293915/zones/us-east1-b/machineTypes/n1-standard-8",
  "displayDevice": {
    "enableDisplay": false
  },
  "metadata": {
    "kind": "compute#metadata",
    "items": [
      {
        "key": "status-config-url",
        "value": "https://runtimeconfig.googleapis.com/v1beta1/projects/dauntless-graph-293915/configs/nvidia-ngc-image-1-config"
      },
      {
        "key": "status-variable-path",
        "value": "status"
      },
      {
        "key": "status-uptime-deadline",
        "value": "300"
      },
      {
        "key": "google-monitoring-enable",
        "value": "0"
      },
      {
        "key": "google-logging-enable",
        "value": "0"
      },
      {
        "key": "startup-script",
        "value": "#!/bin/bash\n\nset -e\n\nDEFAULT_UPTIME_DEADLINE=\"300\"  # 5 minutes\n\nmetadata_value() {\n  curl --retry 5 -sfH \"Metadata-Flavor: Google\" \\\n       \"http://metadata/computeMetadata/v1/$1\"\n}\n\naccess_token() {\n  metadata_value \"instance/service-accounts/default/token\" \\\n  | python3 -c \"import sys, json; print(json.load(sys.stdin)['access_token'])\"\n}\n\nuptime_seconds() {\n  seconds=\"$(cat /proc/uptime | cut -d' ' -f1)\"\n  echo ${seconds%%.*}  # delete floating point.\n}\n\nconfig_url() { metadata_value \"instance/attributes/status-config-url\"; }\ninstance_id() { metadata_value \"instance/id\"; }\nvariable_path() { metadata_value \"instance/attributes/status-variable-path\"; }\nproject_name() { metadata_value \"project/project-id\"; }\nuptime_deadline() {\n  metadata_value \"instance/attributes/status-uptime-deadline\" \\\n      || echo $DEFAULT_UPTIME_DEADLINE\n}\n\nconfig_name() {\n  python3 - $(config_url) <<EOF\nimport sys\nfrom urllib.parse import urlparse\nparsed = urlparse(sys.argv[1])\nprint('/'.join(parsed.path.rstrip('/').split('/')[-4:]))\nEOF\n}\n\nvariable_body() {\n  encoded_value=$(echo \"$2\" | base64)\n  printf '{\"name\":\"%s\", \"value\":\"%s\"}\\n' \"$1\" \"$encoded_value\"\n}\n\npost_result() {\n  var_subpath=$1\n  var_value=$2\n  var_path=\"$(config_name)/variables/$var_subpath/$(instance_id)\"\n\n  curl --retry 5 -sH \"Authorization: Bearer $(access_token)\" \\\n      -H \"Content-Type: application/json\" \\\n      -X POST -d \"$(variable_body \"$var_path\" \"$var_value\")\" \\\n      \"$(config_url)/variables\"\n}\n\npost_success() {\n  post_result \"$(variable_path)/success\" \"${1:-Success}\"\n}\n\npost_failure() {\n  post_result \"$(variable_path)/failure\" \"${1:-Failure}\"\n}\n\n# The contents of initScript are contained within this function.\ncustom_init() (\n  return 0\n)\n\n# The contents of checkScript are contained within this function.\ncheck_success() (\n  return 0\n)\n\ncheck_success_with_retries() {\n  deadline=\"$(uptime_deadline)\"\n  while [ \"$(uptime_seconds)\" -lt \"$deadline\" ]; do\n    message=$(check_success)\n    case $? in\n    0)\n      # Success.\n      return 0\n      ;;\n    1)\n      # Not ready; continue loop\n      ;;\n    *)\n      # Failure; abort.\n      echo $message\n      return 1\n      ;;\n    esac\n\n    sleep 5\n  done\n\n  # The check was not successful within the required deadline.\n  echo \"status check timeout\"\n  return 1\n}\n\ndo_init() {\n  # Run the init script first. If no init script was specified, this\n  # is a no-op.\n  echo \"software-status: initializing...\"\n\n  set +e\n  message=\"$(custom_init)\"\n  result=$?\n  set -e\n\n  if [ $result -ne 0 ]; then\n    echo \"software-status: init failure\"\n    post_failure \"$message\"\n    return 1\n  fi\n}\n\ndo_check() {\n  # Poll for success.\n  echo \"software-status: waiting for software to become ready...\"\n  set +e\n  message=\"$(check_success_with_retries)\"\n  result=$?\n  set -e\n\n  if [ $result -eq 0 ]; then\n    echo \"software-status: success\"\n    post_success\n  else\n    echo \"software-status: failed with message: $message\"\n    post_failure \"$message\"\n  fi\n}\n\n# Run the initialization script synchronously.\ndo_init || exit $?\n\n# The actual software initialization might come after google's init.d\n# script that executes our startup script. Thus, launch this script\n# into the background so that it does not block init and eventually\n# timeout while waiting for software to start.\ndo_check & disown"
      }
    ]
  },
  "tags": {
    "items": [
      "http-server",
      "https-server"
    ]
  },
  "guestAccelerators": [
    {
      "acceleratorCount": 1,
      "acceleratorType": "projects/dauntless-graph-293915/zones/us-east1-b/acceleratorTypes/nvidia-tesla-p100"
    }
  ],
  "disks": [],
  "canIpForward": false,
  "networkInterfaces": [
    {
      "kind": "compute#networkInterface",
      "subnetwork": "projects/dauntless-graph-293915/regions/us-east1/subnetworks/default",
      "accessConfigs": [
        {
          "kind": "compute#accessConfig",
          "name": "External NAT",
          "type": "ONE_TO_ONE_NAT",
          "networkTier": "PREMIUM"
        }
      ],
      "aliasIpRanges": []
    }
  ],
  "description": "",
  "labels": {
    "goog-dm": "nvidia-ngc-image-1"
  },
  "scheduling": {
    "preemptible": false,
    "onHostMaintenance": "TERMINATE",
    "automaticRestart": true,
    "nodeAffinities": []
  },
  "deletionProtection": false,
  "reservationAffinity": {
    "consumeReservationType": "ANY_RESERVATION"
  },
  "serviceAccounts": [
    {
      "email": "669110238078-compute@developer.gserviceaccount.com",
      "scopes": [
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/cloud.useraccounts.readonly",
        "https://www.googleapis.com/auth/cloudruntimeconfig"
      ]
    }
  ],
  "sourceMachineImage": "projects/dauntless-graph-293915/global/machineImages/nvidia-gpu-sc-2",
  "shieldedInstanceConfig": {
    "enableSecureBoot": false,
    "enableVtpm": true,
    "enableIntegrityMonitoring": true
  },
  "confidentialInstanceConfig": {
    "enableConfidentialCompute": false
  }
}

In [62]:
config = '{
  "kind": "compute#instance",
  "name": "nvidia-gpu-sc-1",
  "zone": "projects/dauntless-graph-293915/zones/us-east1-b",
  "minCpuPlatform": "Automatic",
  "machineType": "projects/dauntless-graph-293915/zones/us-east1-b/machineTypes/n1-standard-8",
  "displayDevice": {
    "enableDisplay": false
  },
  "metadata": {
    "kind": "compute#metadata",
    "items": [
      {
        "key": "status-config-url",
        "value": "https://runtimeconfig.googleapis.com/v1beta1/projects/dauntless-graph-293915/configs/nvidia-ngc-image-1-config"
      },
      {
        "key": "status-variable-path",
        "value": "status"
      },
      {
        "key": "status-uptime-deadline",
        "value": "300"
      },
      {
        "key": "google-monitoring-enable",
        "value": "0"
      },
      {
        "key": "google-logging-enable",
        "value": "0"
      },
      {
        "key": "startup-script",
        "value": "#!/bin/bash\n\nset -e\n\nDEFAULT_UPTIME_DEADLINE=\"300\"  # 5 minutes\n\nmetadata_value() {\n  curl --retry 5 -sfH \"Metadata-Flavor: Google\" \\\n       \"http://metadata/computeMetadata/v1/$1\"\n}\n\naccess_token() {\n  metadata_value \"instance/service-accounts/default/token\" \\\n  | python3 -c \"import sys, json; print(json.load(sys.stdin)['access_token'])\"\n}\n\nuptime_seconds() {\n  seconds=\"$(cat /proc/uptime | cut -d' ' -f1)\"\n  echo ${seconds%%.*}  # delete floating point.\n}\n\nconfig_url() { metadata_value \"instance/attributes/status-config-url\"; }\ninstance_id() { metadata_value \"instance/id\"; }\nvariable_path() { metadata_value \"instance/attributes/status-variable-path\"; }\nproject_name() { metadata_value \"project/project-id\"; }\nuptime_deadline() {\n  metadata_value \"instance/attributes/status-uptime-deadline\" \\\n      || echo $DEFAULT_UPTIME_DEADLINE\n}\n\nconfig_name() {\n  python3 - $(config_url) <<EOF\nimport sys\nfrom urllib.parse import urlparse\nparsed = urlparse(sys.argv[1])\nprint('/'.join(parsed.path.rstrip('/').split('/')[-4:]))\nEOF\n}\n\nvariable_body() {\n  encoded_value=$(echo \"$2\" | base64)\n  printf '{\"name\":\"%s\", \"value\":\"%s\"}\\n' \"$1\" \"$encoded_value\"\n}\n\npost_result() {\n  var_subpath=$1\n  var_value=$2\n  var_path=\"$(config_name)/variables/$var_subpath/$(instance_id)\"\n\n  curl --retry 5 -sH \"Authorization: Bearer $(access_token)\" \\\n      -H \"Content-Type: application/json\" \\\n      -X POST -d \"$(variable_body \"$var_path\" \"$var_value\")\" \\\n      \"$(config_url)/variables\"\n}\n\npost_success() {\n  post_result \"$(variable_path)/success\" \"${1:-Success}\"\n}\n\npost_failure() {\n  post_result \"$(variable_path)/failure\" \"${1:-Failure}\"\n}\n\n# The contents of initScript are contained within this function.\ncustom_init() (\n  return 0\n)\n\n# The contents of checkScript are contained within this function.\ncheck_success() (\n  return 0\n)\n\ncheck_success_with_retries() {\n  deadline=\"$(uptime_deadline)\"\n  while [ \"$(uptime_seconds)\" -lt \"$deadline\" ]; do\n    message=$(check_success)\n    case $? in\n    0)\n      # Success.\n      return 0\n      ;;\n    1)\n      # Not ready; continue loop\n      ;;\n    *)\n      # Failure; abort.\n      echo $message\n      return 1\n      ;;\n    esac\n\n    sleep 5\n  done\n\n  # The check was not successful within the required deadline.\n  echo \"status check timeout\"\n  return 1\n}\n\ndo_init() {\n  # Run the init script first. If no init script was specified, this\n  # is a no-op.\n  echo \"software-status: initializing...\"\n\n  set +e\n  message=\"$(custom_init)\"\n  result=$?\n  set -e\n\n  if [ $result -ne 0 ]; then\n    echo \"software-status: init failure\"\n    post_failure \"$message\"\n    return 1\n  fi\n}\n\ndo_check() {\n  # Poll for success.\n  echo \"software-status: waiting for software to become ready...\"\n  set +e\n  message=\"$(check_success_with_retries)\"\n  result=$?\n  set -e\n\n  if [ $result -eq 0 ]; then\n    echo \"software-status: success\"\n    post_success\n  else\n    echo \"software-status: failed with message: $message\"\n    post_failure \"$message\"\n  fi\n}\n\n# Run the initialization script synchronously.\ndo_init || exit $?\n\n# The actual software initialization might come after google's init.d\n# script that executes our startup script. Thus, launch this script\n# into the background so that it does not block init and eventually\n# timeout while waiting for software to start.\ndo_check & disown"
      }
    ]
  },
  "tags": {
    "items": [
      "http-server",
      "https-server"
    ]
  },
  "guestAccelerators": [
    {
      "acceleratorCount": 1,
      "acceleratorType": "projects/dauntless-graph-293915/zones/us-east1-b/acceleratorTypes/nvidia-tesla-p100"
    }
  ],
  "disks": [{"type": "PERSISTENT",
   "mode": "READ_WRITE",
   "source": "https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/disks/gpu-isness-server-sc-p100-2",
   "deviceName": "nvidia-ngc-image-vm-tmpl-boot-disk",
   "index": "0",
   "boot": "True",
   "autoDelete": "True",
   "diskSizeGb": "32",
   "kind": "compute#attachedDisk"}],
  "canIpForward": "false",
  "networkInterfaces": [
    {
      "kind": "compute#networkInterface",
      "subnetwork": "projects/dauntless-graph-293915/regions/us-east1/subnetworks/default",
      "accessConfigs": [
        {
          "kind": "compute#accessConfig",
          "name": "External NAT",
          "type": "ONE_TO_ONE_NAT",
          "networkTier": "PREMIUM"
        }
      ],
      "aliasIpRanges": []
    }
  ],
  "description": "",
  "labels": {
    "goog-dm": "nvidia-ngc-image-1"
  },
  "scheduling": {
    "preemptible": false,
    "onHostMaintenance": "TERMINATE",
    "automaticRestart": true,
    "nodeAffinities": []
  },
  "deletionProtection": false,
  "reservationAffinity": {
    "consumeReservationType": "ANY_RESERVATION"
  },
  "serviceAccounts": [
    {
      "email": "669110238078-compute@developer.gserviceaccount.com",
      "scopes": [
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/cloud.useraccounts.readonly",
        "https://www.googleapis.com/auth/cloudruntimeconfig"
      ]
    }
  ],
  "sourceMachineImage": "projects/dauntless-graph-293915/global/machineImages/nvidia-gpu-sc-2",
  "shieldedInstanceConfig": {
    "enableSecureBoot": false,
    "enableVtpm": true,
    "enableIntegrityMonitoring": true
  },
  "confidentialInstanceConfig": {
    "enableConfidentialCompute": false
  }
}'

SyntaxError: EOL while scanning string literal (<ipython-input-62-606b6dd31f1a>, line 1)

In [95]:
def create_instance(region, duration, simulation=None, topology=None, trajectory=None):
    zone = gcp.get_zone_for_region(region)

    machine_type = 'n1-standard-1'
    gpu_type = 'nvidia-tesla-p100'
    disk_size_gb = '32'

    name = 'anuma-isness-{}'.format(utils.generate_short_id())
    metadata = [
        { 'key': 'google-logging-enabled', 'value': 'true' },
        { 'key': 'duration', 'value': duration},
        { 'key': 'startup-script', 'value': '#!/bin/bash\n'}
    ]

    config = {
        "name": name,
         "kind": "compute#instance",
         "zone": "projects/dauntless-graph-293915/zones/us-east1-b",
        "minCpuPlatform": "Automatic",
        "machineType": "projects/dauntless-graph-293915/zones/us-east1-b/machineTypes/n1-standard-8",
        "displayDevice": {
            "enableDisplay": "false"
        },
        "guestAccelerators": [
            { "acceleratorCount": 1,
            "acceleratorType": "projects/dauntless-graph-293915/zones/us-east1-b/acceleratorTypes/nvidia-tesla-p100" }
        ],
        "sourceMachineImage": "projects/dauntless-graph-293915/global/machineImages/nvidia-gpu-sc-2",
        "serviceAccounts": [
            { "email": "669110238078-compute@developer.gserviceaccount.com",
            "scopes": [
                "https://www.googleapis.com/auth/cloud-platform",
                "https://www.googleapis.com/auth/cloud.useraccounts.readonly",
                "https://www.googleapis.com/auth/cloudruntimeconfig"
            ]
        }
        ],
        "scheduling": {
            "preemptible": "false",
            "onHostMaintenance": "TERMINATE",
            "automaticRestart": "true",
            "nodeAffinities": [] },
        "deletionProtection": "false",
        "reservationAffinity": {
            "consumeReservationType": "ANY_RESERVATION" },
        "disks": [{"type": "PERSISTENT",
            "mode": "READ_WRITE",
            'initializeParams': {
                'diskType': 'projects/{}/zones/{}/diskTypes/pd-standard'.format(PROJECT, zone),
                'sourceImage': 'projects/nvidia-ngc-public/global/images/nvidia-gpu-cloud-image-20200629',
                'diskSizeGb': disk_size_gb
            },
            "deviceName": "nvidia-ngc-image-vm-tmpl-boot-disk",
            "index": "0",
            "boot": "True",
            "autoDelete": "True",
            "diskSizeGb": "32",
            "kind": "compute#attachedDisk"}],
            "canIpForward": "false",
        "networkInterfaces": [
            {
            "kind": "compute#networkInterface",
            "subnetwork": "projects/dauntless-graph-293915/regions/us-east1/subnetworks/default",
            "accessConfigs": [
                {
                "kind": "compute#accessConfig",
                "name": "External NAT",
                "type": "ONE_TO_ONE_NAT",
                "networkTier": "PREMIUM"
                }
            ]
            }
        ]
    }
    response = gcp.get_compute_client().instances().insert(project=PROJECT, zone=zone, body=config).execute()
    response['instanceName'] = name
    return response

In [96]:
create_instance(region, duration, simulation=None, topology=None, trajectory=None)

{'id': '2264659484805103208',
 'name': 'operation-1605058694744-5b3cad95808b0-40f0412e-0ab3d6bd',
 'zone': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b',
 'operationType': 'insert',
 'targetLink': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/instances/anuma-isness-ksoeak',
 'targetId': '6070837928899575401',
 'status': 'RUNNING',
 'user': '669110238078-compute@developer.gserviceaccount.com',
 'progress': 0,
 'insertTime': '2020-11-10T17:38:15.480-08:00',
 'startTime': '2020-11-10T17:38:15.483-08:00',
 'selfLink': 'https://www.googleapis.com/compute/v1/projects/dauntless-graph-293915/zones/us-east1-b/operations/operation-1605058694744-5b3cad95808b0-40f0412e-0ab3d6bd',
 'kind': 'compute#operation',
 'instanceName': 'anuma-isness-ksoeak'}